# 线性回归的简洁实现

In [1]:
import torch
import numpy as np
from d2l import torch as d2l
from torch.utils import data

## 生成数据集

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[tensor([[ 0.8488,  0.9038],
         [ 0.4985, -0.1811],
         [-0.4726,  1.3105],
         [ 2.0888, -0.4593],
         [ 1.1183, -0.4287],
         [ 0.0942, -1.3193],
         [ 0.5109, -1.8160],
         [-0.0985, -0.7091],
         [ 0.5021,  0.6062],
         [ 1.9745, -1.1453]]),
 tensor([[ 2.8387],
         [ 5.7956],
         [-1.1901],
         [ 9.9254],
         [ 7.8990],
         [ 8.8787],
         [11.3897],
         [ 6.4193],
         [ 3.1386],
         [12.0506]])]

## 定义模型

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## 初始化参数

In [7]:
net[0].bias.data.fill_(0)
net[0].weight.data.normal_(0, 0.01)

tensor([[0.0311, 0.0017]])

## 定义损失函数

In [8]:
loss = nn.MSELoss()

## 定义优化算法

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

* 通过调用 `net(X)` 生成预测并计算损失 `l`（正向传播）。
* 通过进行反向传播来计算梯度。
* 通过调用优化器来更新模型参数。

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {1:f}') 

epoch 1, loss 1.000000
epoch 2, loss 1.000000
epoch 3, loss 1.000000


In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0005,  0.0004])
b的估计误差： tensor([0.0005])


## 练习
1. 如果我们用 `nn.MSELoss()` 替换 `nn.MSELoss(reduction='sum')`，为了使代码的行为相同，需要怎么更改学习速率？为什么？
1. 查看 PyTorch 文档，了解提供了哪些损失函数和初始化方法。用Huber损失来代替。
1. 你如何访问 `net[0].weight` 的梯度？